In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Packages.ClusteringHelper as ch
import numpy as np
from collections import Counter
import string
from nltk.corpus import stopwords

In [3]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [5]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
docs = ents_data['documents'].values
word_indexes = ents_data['word_indexes'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

#### Gensim word2vec

In [6]:
lowercase = True
remove_stopwords = True
remove_numbers = True
remove_symbols = True

text_preprocessed = text
stop_words = set(stopwords.words("english"))

if remove_symbols:
    text_preprocessed = [x.translate(str.maketrans('', '', string.punctuation)) for x in text_preprocessed]

# rimuovo numeri e punteggiatura
if lowercase:
    text_splitted_training = [x.lower().split() for x in text_preprocessed]
    text_splitted = [x.lower().split() for x in text]
else:
    text_splitted_training = [x.split() for x in text_preprocessed]
    text_splitted = [x.split() for x in text]

if remove_numbers:
    text_splitted_training = [[y for y in x if not y.isnumeric()] for x in text_splitted_training]

if remove_stopwords:
    text_splitted_training = [[y for y in x if y.lower() not in stop_words] for x in text_splitted_training]

Model selection

In [7]:
# create the model
import os
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

import os
from gensim.models import KeyedVectors

model_path = "D:\Sgmon\Documents\Magistrale\TESI\gensim-data"
"""
sg 0 = CBOW
sh 1 = SKIPGRAM
"""
model = Word2Vec(text_splitted_training, min_count=5, workers=cpu_count(), window=1, sg=0, size=25)

# or select it
#
# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'word2vec-google-news-300', "word2vec-google-news-300.gz"),
#                                                        binary=True)

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'glove-twitter-25', 'glove-twitter-25.gz'))

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-50.gz'))


# model = KeyedVectors.load_word2vec_format(
#     os.path.join(model_path, 'glove-wiki-gigaword-300', "glove-wiki-gigaword-300.txt"))

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'fasttext-wiki-news-subwords-300', "fasttext-wiki-news-subwords-300.gz"),
#                                                        binary=False)

In [8]:
lev_cluster_dict = []
windows = 25
doc_windows = 20
for i in range(len(mentions)):
    lev_cluster_dict.append(
        (mentions[i], entities[i], docs[i], word_indexes[i],
         ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows)))

In [21]:
windows = 25
doc_windows = 20
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# from hdbscan import HDBSCAN

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='manhattan', distance_threshold=0.01, linkage="single")
# clusterizator = DBSCAN(min_samples=0, metric="cosine", n_jobs=-1, eps=0.1)
# clusterizator = HDBSCAN(min_cluster_size=2, metric="euclidean", min_samples=1, cluster_selection_epsilon=0.4)

contexts_vectorized = ch.calculate_context_vector(model, lev_cluster_dict, text_splitted, doc_window=doc_windows,
                                                      doc_context_alpha=0.3, word_context_aplha=.7)
if len(contexts_vectorized) > 1:
    cluster = clusterizator.fit_predict(contexts_vectorized)
else:
    cluster = np.zeros(1)
    # print(cluster_all)
    # break

In [22]:
cluster_dict = {}
for i, cl in enumerate(cluster):
    try:
        cluster_dict[cl].append(entities[i])
    except:
        cluster_dict[cl] = [entities[i]]

In [23]:
for x in cluster_dict:
    cluster_dict[x] = Counter(cluster_dict[x])

In [24]:
cluster_dict

{5573: Counter({'Germany': 1, 'United_Kingdom': 1, 'Brussels': 1}),
 10942: Counter({'European_Commission': 1}),
 12603: Counter({'Germany': 1}),
 9409: Counter({'United_Kingdom': 1}),
 10918: Counter({'Germany': 1}),
 14765: Counter({'European_Union': 1}),
 11143: Counter({'United_Kingdom': 1}),
 11038: Counter({'European_Commission': 1}),
 13573: Counter({'European_Union': 1}),
 3087: Counter({'United_Kingdom': 1, 'France': 1}),
 10014: Counter({'Bovine_spongiform_encephalopathy': 1}),
 13333: Counter({'Spain': 1}),
 3204: Counter({'France': 1, 'United_Kingdom': 1}),
 16153: Counter({'Bovine_spongiform_encephalopathy': 1}),
 10935: Counter({'United_Kingdom': 1}),
 10307: Counter({'Germany': 1}),
 10136: Counter({'United_Kingdom': 1}),
 13281: Counter({'Europe': 1}),
 9726: Counter({'Germany': 1}),
 11792: Counter({'Bonn': 1}),
 14161: Counter({'United_Kingdom': 1}),
 12607: Counter({'Germany': 1}),
 14999: Counter({'United_Kingdom': 1}),
 14335: Counter({'United_Kingdom': 1}),
 3274: